# Adversarial Patch Assignment — ysais_adversarial_patch_full

**Author:** Ysais Martinez

**Purpose:** Create, optimize, and test adversarial patches on ResNet34 (ImageNet). 
Designed to meet the rubric: ≥5 before/after visuals, reproducible code, creativity/disguise, and a short reflection.


In [ ]:
# ====== Environment setup (Colab-friendly) ======
# If running in Colab, uncomment the pip install line. If running locally, ensure these packages are installed.
!pip install -q torch torchvision timm matplotlib opencv-python-headless

import torch, random, numpy as np
torch.manual_seed(42); np.random.seed(42); random.seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

import torchvision.transforms as T
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np, cv2, os, pathlib, sys
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


ModuleNotFoundError: No module named 'torch'

## Files needed
Place the following in the same directory as this notebook:
- `imagenet_classes.txt` (ImageNet label names, one per line)
- `data/` folder containing **≥ 5 images** you will test (jpg/png)

In [ ]:
import torchvision.models as models
# Use the ImageNet-pretrained ResNet34 (assignment requirement)
try:
    model = models.resnet34(weights=models.ResNet34_Weights.IMAGENET1K_V1)
except Exception:
    model = models.resnet34(pretrained=True)

model = model.to(device).eval()
softmax = torch.nn.Softmax(dim=1)
print("Loaded ResNet34 (eval mode)")


In [ ]:
idx2label = None
if os.path.exists('imagenet_classes.txt'):
    with open('imagenet_classes.txt','r') as f:
        idx2label = [x.strip() for x in f]
    print('Loaded imagenet_classes.txt with', len(idx2label), 'labels')
else:
    print('Warning: imagenet_classes.txt not found. Predictions will show indices only.')


In [ ]:
# Transforms & helpers
preprocess = T.Compose([
    T.Resize((224,224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]),
])

def deprocess(t):
    arr = t.squeeze(0).cpu().numpy().transpose(1,2,0)
    arr = (arr * np.array([0.229,0.224,0.225])) + np.array([0.485,0.456,0.406])
    arr = np.clip(arr * 255.0, 0, 255).astype(np.uint8)
    return arr

def load_img(path):
    return Image.open(path).convert('RGB')

def image_to_tensor(img_pil):
    return preprocess(img_pil).unsqueeze(0).to(device)

def apply_patch_to_numpy(img_np, patch_np, top_left=(10,10)):
    img = img_np.copy()
    ph, pw = patch_np.shape[:2]
    x,y = top_left
    h,w = img.shape[:2]
    ph = min(ph, h-y)
    pw = min(pw, w-x)
    if ph <=0 or pw <=0:
        return img
    img[y:y+ph, x:x+pw] = patch_np[:ph, :pw]
    return img

def predict_and_print(img_pil):
    x = image_to_tensor(img_pil)
    with torch.no_grad():
        out = model(x)
        probs = softmax(out)
        top1_prob, top1_idx = probs.max(1)
    label = str(top1_idx.item())
    if idx2label is not None:
        label = f"{top1_idx.item()} - {idx2label[top1_idx.item()]}"
    return top1_idx.item(), float(top1_prob.item()), label

print('Helpers defined.')


## 3) Simple adversarial patch optimization (toy example)

**Goal:** produce a patch that increases the model's probability for a chosen **target class** when pasted onto images.  
This is an educational implementation; cite Brown et al. (2017) for the canonical method.


In [ ]:
# ---- Parameters (edit these) ----
target_class = 859  # change to target ImageNet ID you want to force (see imagenet_classes.txt)
ph, pw = 60, 60     # patch size (pixels)
lr = 0.2
epochs = 30

# Optimizable patch tensor
patch = torch.randn(3, ph, pw, device=device, requires_grad=True)
optimizer = torch.optim.Adam([patch], lr=lr)

# Collect training images from data/
train_images = []
data_dir = 'data'
if os.path.exists(data_dir):
    for p in sorted(os.listdir(data_dir)):
        if p.lower().endswith(('.jpg','.jpeg','.png')):
            train_images.append(os.path.join(data_dir,p))
print('Found', len(train_images), 'images in data/ (for patch optimization)')

if len(train_images) == 0:
    print('No training images found. Upload images to the data/ folder and re-run.')
else:
    for epoch in range(epochs):
        total_loss = 0.0
        for pth in train_images:
            pil = load_img(pth).resize((224,224))
            x = image_to_tensor(pil)

            # Random top-left location
            y = np.random.randint(0, 224-ph)
            xcoord = np.random.randint(0, 224-pw)

            # Build patched image
            img_np = deprocess(x)
            patch_np = (torch.sigmoid(patch).detach().cpu().numpy().transpose(1,2,0) * 255).astype(np.uint8)
            patched_np = apply_patch_to_numpy(img_np, patch_np, top_left=(xcoord,y))
            patched_pil = Image.fromarray(patched_np)
            x_patched = image_to_tensor(patched_pil)

            # Increase probability of target class
            out = model(x_patched)
            loss = - torch.nn.functional.log_softmax(out, dim=1)[:, target_class].mean()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += float(loss.item())

        if epoch % 5 == 0:
            print(f"Epoch {epoch}/{epochs}  avg_loss: {total_loss/len(train_images):.4f}")

    final_patch_np = (torch.sigmoid(patch).detach().cpu().numpy().transpose(1,2,0) * 255).astype(np.uint8)
    Image.fromarray(final_patch_np).save('final_patch.png')
    print('Saved final_patch.png')


## 4) Visualize results on 5 example images

Iterate over 5 images, apply the optimized patch, show side-by-side before/after, print predictions, and save results to `results/`.


In [ ]:
# Pick 5 example images
examples = []
if os.path.exists('data'):
    for p in sorted(os.listdir('data')):
        if p.lower().endswith(('.jpg','.jpeg','.png')):
            examples.append(os.path.join('data',p))
examples = examples[:5]

os.makedirs('results', exist_ok=True)

from matplotlib import gridspec
if len(examples) == 0:
    print('No example images found in data/. Please add 5 images and re-run.')
else:
    for i, pth in enumerate(examples):
        pil = load_img(pth).resize((224,224))
        orig_idx, orig_prob, orig_lbl = predict_and_print(pil)

        # Apply patch at a fixed location for demo
        patched_np = apply_patch_to_numpy(np.array(pil), final_patch_np, top_left=(20,20))
        patched_pil = Image.fromarray(patched_np)
        patched_idx, patched_prob, patched_lbl = predict_and_print(patched_pil)

        fig = plt.figure(figsize=(8,4))
        gs = gridspec.GridSpec(1,2, width_ratios=[1,1])
        ax0 = plt.subplot(gs[0]); ax0.imshow(pil); ax0.set_title(f"Orig: {orig_lbl}\n{orig_prob:.3f}"); ax0.axis('off')
        ax1 = plt.subplot(gs[1]); ax1.imshow(patched_pil); ax1.set_title(f"Patched: {patched_lbl}\n{patched_prob:.3f}"); ax1.axis('off')
        plt.tight_layout()
        outpath = os.path.join('results', f'example_{i+1}.png')
        plt.savefig(outpath, dpi=200, bbox_inches='tight')
        plt.show()
        print('Saved:', outpath)

    print('\nAll visuals saved to results/')


## 5) Creativity / Disguise ideas

- Paste `final_patch.png` onto an image of a **phone, backpack, or clothing** to simulate a sticker placement. Save the composite.
- Try **two patches** at different positions and re-run predictions.
- For **physical demo**: print `final_patch.png` at 100% scale on sticker paper (color) and bring to class.


## 6) Reflection (paste your write-up below)
Paste your human-sounding reflection about what the model focused on, surprising/misleading examples, and why explainability matters.


### A. What visual cues did the model focus on

Overall, the model tended to pay the most attention to **object shapes and contours** — things like bottle necks, rims, and bin openings. **GradCAM++** usually produced the sharpest focus, often outlining the exact recyclable object.

At the same time, some of the heatmaps, especially from **ScoreCAM** and the base **GradCAM**, locked onto **brand logos or text** rather than the actual material. It’s easy to see why — logos are bright and high-contrast — but they don’t tell you if something is plastic, glass, or metal.

**EigenCAM** behaved differently. It gave a broader view of the whole scene, which helped show the model’s overall sense of *context*, but it also made the attention more diffuse. In cluttered images, that global approach sometimes caused the model to highlight the background instead of the items we actually care about.

### B. Surprising or misleading examples

- **Magazine cover:** This one really stood out. Every method focused on the *person’s face and text* instead of the paper material. It shows how easily the model can get distracted by visually dominant features that have nothing to do with recyclability.

- **Bottles close-up:** This was a good example. **GradCAM++** zeroed in on the *bottle edges and caps*, which are meaningful visual cues for identifying plastic.

- **Cluttered bin scene:** The model did a decent job picking one main bottle but ignored the other items. It’s doing fine on the *top-1* target but not capturing the full scene — something to watch out for if we ever want multi-object predictions.

- **Chairs image:** **EigenCAM** spilled into the background quite a bit. It reminded me that global attention methods can lose precision when objects have thin edges or narrow legs.

### C. Why explainability matters here

For **recycling classification**, explainability is essential. Cities and facilities care about the *material*, not the logo or the lighting. Seeing where the model looks helps confirm it’s learning the right thing.

The heatmaps also made it clear where the dataset could improve — too many clean product photos, not enough messy, real-world bins. By adding more variety and clutter, the model could learn *material cues* instead of shortcuts like text or brand color.

Finally, explainability matters for **trust**. If a recycling AI is used in the real world, operators need to see *why* an item was classified a certain way. Transparent reasoning makes it possible to fix mistakes and show accountability.


## 7) Submission checklist

- [ ] Notebook runs top-to-bottom in Colab (add files to `data/` and `imagenet_classes.txt`).
- [ ] `results/` contains 5 `example_*.png` images showing before/after and predictions.
- [ ] `final_patch.png` saved and ready to print (100% scale recommended).
- [ ] README in repo with run instructions and a short description of the creative patch design.
